# Goal
We want $$0 = \dot{\hat{\rho}} = \mathcal{L}\hat{\rho} = -i\left[\hat H, \hat \rho\right]+\sum_j \frac{\gamma_j}{2}\left[2\hat L_j\hat\rho\hat L_j^\dagger - \left\{\hat L_j^\dagger\hat L_j, \hat\rho\right\}\right]$$
In terms of probabilities for POVM, it is equavalent to
$$0=\dot p_a = \sum_b p_b(-i{A_a}^b+{B_a}^b)$$
where
$${A_a}^b = \text{tr}\left(HN^bM_a\right)-\text{tr}\left(N^bHM_a\right) = \sum_{r,s,t}\left(H_{rs}{N^b}_{st}M_{atr}-{N^b}_{rs}H_{st}M_{atr}\right)$$
\begin{align*}
{B_a}^b  &= \sum_j\left[\text{tr}\left(L_jN^bL_j^\dagger M_a\right) -\frac{1}{2}\text{tr}\left(L_j^\dagger L_j N^b M_a\right)-\frac{1}{2}\text{tr}\left(N^b L_j^\dagger L_j M_a\right)\right] \\
&= \sum_{j, r,s,t,u}L_{jrs}{N^b}_{st}L_{jut}^* M_{aur}-\frac{1}{2}L_{jsr}^* L_{jst} {N^b}_{tu} M_{aur} - \frac{1}{2}{N^b}_{rs} L_{jts}^* L_{jtu} M_{aur}
\end{align*}

In [1]:
import os
os.environ["OMP_NUM_THREADS"] = '16'
import numpy as np
import scipy as sp
from scipy import optimize
import matplotlib.pyplot as plt
import time
# local packages
from model import utils
from utils.POVM import *
from utils.ncon import *

Pauli Matrices

In [2]:
sigma_x = np.array([[0, 1], [1, 0]], dtype = 'complex128')
sigma_y = np.array([[0, -1j], [1j, 0]], dtype = 'complex128')
sigma_z = np.array([[1, 0], [0, -1]], dtype = 'complex128')
identity = np.array([[1, 0], [0, 1]], dtype = 'complex128')

Some parameters

In [5]:
num_latt = 4
coup_stren = 2
povm_basis = 'Tetra'  # 4Pauli, Tetra, Tetra_pos
method = 'direct' # direct, gradient

Generate POVM

In [6]:
povm = POVM(POVM = povm_basis, Number_qubits = num_latt)
povm.construct_Nframes()
Mn = povm.Mn # M_a
Ntn = povm.Ntn # N^b

In [7]:
print(Mn.shape)
print(Ntn.shape)

(256, 16, 16)
(256, 16, 16)


$$ H = \frac{V}{4}\sum_{\langle j, l\rangle}\hat\sigma_j^z\hat\sigma_l^z+\frac{g}{2}\sum_j\hat\sigma_j^x$$
where $\hat\sigma_j^\alpha$ means the Pauli matrix acting on the $j$-th site.
$$ \hat L_j^{(z)} = \hat\sigma_j^- = \frac{1}{2}\left(\hat\sigma_j^x-i\hat\sigma_j^y\right)$$

In [ ]:
# make tensor product of two matrices (copied over from povm code, 
# but I didn't reverse the order because later I shall use this function
# in a different order.)
def tensorproduct(matrix1, matrix2):
    dim = matrix1.shape[0] * matrix2.shape[0]
    # didn't reverse the order here
    return np.tensordot(matrix1, matrix2, axes = 0).swapaxes(1, 2).reshape(dim, dim)

In [ ]:
# generate hamiltonion (rank 2 tensor)
def gen_H(num, V, g):
    hamiltonion = np.zeros((2**num, 2**num), dtype = 'complex128')
    for i in range(num):
        term1 = np.ones((1, 1), dtype = 'complex128')
        term2 = np.ones((1, 1), dtype = 'complex128')
        for j in range(num): 
            if j == i or j == i + 1 or (i == num - 1 and j == 0): # wallpaper(cyclic) bondary condition
                term1 = tensorproduct(term1, sigma_z) # order here not the same as in POVM
                #print(i, j)
                #print(term1)
            else:
                term1 = tensorproduct(term1, identity) # order here not the same as in POVM
            if j == i:
                term2 = tensorproduct(term2, sigma_x) # order here not the same as in POVM
            else:
                term2 = tensorproduct(term2, identity) # order here not the same as in POVM
        term1 = term1 * V / 4
        #print(term1)
        term2 = term2 * g / 2
        hamiltonion += term1 + term2
    return hamiltonion
                

In [ ]:
# generate L (rank 3 tensor)
def gen_L(num):
    L_list = []
    for i in range(num):
        term = np.ones((1, 1), dtype = 'complex128')
        for j in range(num):
            if j == i:
                term = tensorproduct(term, (sigma_x - 1j * sigma_y) / 2)
            else:
                term = tensorproduct(term, identity)
        L_list.append(term)
    return np.array(L_list)
            

$${A_a}^b = \sum_{r,s,t}\left(H_{rs}{N^b}_{st}M_{atr}-{N^b}_{rs}H_{st}M_{atr}\right)$$
$${B_a}^b= \sum_{j, r,s,t,u}L_{jrs}{N^b}_{st}L_{jut}^* M_{aur}-\frac{1}{2}L_{jsr}^* L_{jst} {N^b}_{tu} M_{aur} - \frac{1}{2}{N^b}_{rs} L_{jts}^* L_{jtu} M_{aur}$$

In [8]:
def gen_A(H, M, N):
    term1 = ncon((H, N, M), ([1,2],[-2,2,3],[-1,3,1]))
    term2 = ncon((N, H, M), ([-2,1,2],[2,3],[-1,3,1]))
    return term1 - term2

In [9]:
def gen_B(L, M, N): 
    term1 = ncon((L, N, np.conj(L), M), ([1,2,3],[-2,3,4],[1,5,4],[-1,5,2]))
    term2 = ncon((np.conj(L), L, N, M), ([1,3,2],[1,3,4],[-2,4,5],[-1,5,2]))
    term3 = ncon((N, np.conj(L), L, M), ([-2,2,3],[1,4,3],[1,4,5],[-1,5,2]))
    return term1 - term2/2 - term3 / 2

In [ ]:
g_list = np.linspace(0, 4, 17)


def direct():
    p_list = []
    for g in g_list:
        start_time = time.time()
        H = gen_H(num_latt, coup_stren, g)
        L = gen_L(num_latt)
        A = gen_A(H, Mn, Ntn)
        B = gen_B(L, Mn, Ntn)
        C = (-1j * A + B).real
        p = sp.linalg.null_space(C)
        p /= np.sum(p)
        p_list.append(p.flatten())
        print(g, time.time() - start_time)
    return p_list

def gradient():
    p_list = []
    for g in g_list:
        start_time = time.time()
        initial_parameters = np.random.random(4 ** num_latt - 1)
        H = gen_H(num_latt, coup_stren, g)
        L = gen_L(num_latt)
        A = gen_A(H, Mn, Ntn)
        B = gen_B(L, Mn, Ntn)
        C = (-1j * A + B).real
        def loss(parameters):
            prob = np.append(parameters, 0)
            prob[-1] = 1 - np.sum(prob)
            prob_dot = C @ prob
            return np.linalg.norm(prob_dot)
        res = optimize.minimize(loss, initial_parameters, options={'gtol':1e-12})
        optimized_parameters = res.x
        loss_value = res.fun
        p = np.append(optimized_parameters, 0)
        p[-1] = 1 - np.sum(p)
        p_list.append(p)
        print(g, loss_value, time.time() - start_time)
    return p_list

p_list = []
if method == 'direct':
    p_list = direct()
elif method == 'gradient':
    p_list = gradient()

In [ ]:
p_list

In [ ]:
rho_list = []
for p in p_list:
    #print(p.shape)
    #print(Ntn.shape)
    rho = ncon((p, Ntn), ([1],[1,-1,-2]))
    rho_list.append(rho)
rho_list

In [ ]:
def gen_big_sigma(sigma, num):
    big_sigma = np.zeros((2**num, 2**num), dtype = 'complex128')
    for i in range(num):
        current_sigma = np.array([1])
        for j in range(num):
            if j == i:
                current_sigma = tensorproduct(current_sigma, sigma)
            else:
                current_sigma = tensorproduct(current_sigma, identity)
            #print(i, j, current_sigma)
        big_sigma += current_sigma
    big_sigma /= num
    return big_sigma

In [ ]:
gen_big_sigma(sigma_z, 4)

In [ ]:
big_sigma_x = gen_big_sigma(sigma_x, num_latt)
big_sigma_y = gen_big_sigma(sigma_y, num_latt)
big_sigma_z = gen_big_sigma(sigma_z, num_latt)

In [ ]:
big_sigma_x

In [ ]:
sigma_x_expect = list(map(lambda r: np.trace(r @ big_sigma_x), rho_list))
sigma_y_expect = list(map(lambda r: np.trace(r @ big_sigma_y), rho_list))
sigma_z_expect = list(map(lambda r: np.trace(r @ big_sigma_z), rho_list))

In [ ]:
plt.figure()
plt.plot(g_list, sigma_x_expect)
plt.xlabel(r'$g / \gamma$')
plt.ylabel(r'$\langle\sigma_x\rangle$')
plt.show()
plt.figure()
plt.plot(g_list, sigma_y_expect)
plt.xlabel(r'$g / \gamma$')
plt.ylabel(r'$\langle\sigma_y\rangle$')
plt.show()
plt.figure()
plt.plot(g_list, sigma_z_expect)
plt.xlabel(r'$g / \gamma$')
plt.ylabel(r'$\langle\sigma_z\rangle$')
plt.show()

In [ ]:
rho_list2 = np.load('rho_list.npy')

In [ ]:
rho_list1 = np.array(rho_list)

In [ ]:
np.linalg.norm(rho_list2-rho_list1)